## Exercise: Grover's Algorithms
Brief description of Grover's algorithm (maybe on the slides?)

Package required:

In [ ]:
!pip install qibo
!pip install qibojit

Check the version

In [ ]:
import qibo
import qibojit
print(qibo.__version__)
print(qibojit.__version__)

Import modules

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from qibo import models, gates, set_backend
set_backend("numpy")

### Exercise: Grover's algorithm using Qibo

We already saw in the slides that the Grover's algorithm is divided into 3 different parts
* Create superposition
* Create oracle
* Create diffusion operator

Write quantum circuit that implements Grover's algorithm for a generic initial state.

*Hint: Start by the case 11...1*

#### Step 1: creating the superposition
We need to write a circuit that will implement the superposition of all possible states.

In [ ]:
def create_superposition(nqubits):
    """Create circuit for superposition"""
    superposition = models.Circuit(nqubits+1)
    superposition.add([gates.H(i) for i in range(nqubits)])
    # Add ancilla qubit
    superposition.add(gates.X(nqubits))
    superposition.add(gates.H(nqubits))

    return superposition

You can check the gates in the circuit by drawing it

In [ ]:
superposition = create_superposition(3)
print(superposition.draw())

To check whether the circuit works as expected you can visualize the amplitudes using the function `plot_amplitude`.

In [ ]:
def plot_amplitudes(amplitudes):
    """Plot amplitudes of the quantum circuit"""
    amplitudes = amplitudes.state()
    states = []
    amp = []
    for i in range(int(len(amplitudes)/2)):
        states.append("{0:0{bits}b}".format(i, bits=int(np.log2(len(amplitudes)/2))))
    for i in range(0, len(amplitudes), 2):
        amp.append((1/np.sqrt(2))*(np.real(amplitudes[i])-np.real(amplitudes[i+1])))
    fig = plt.figure(figsize = (18,6))
    width = 0.5
    plt.title('Amplitudes', fontdict={'fontsize': 14})
    plt.xlabel('state', fontsize=14)
    plt.ylabel('magnitude', fontsize=14)
    plt.ylim(-1.1,1.1)
    plt.bar(states, amp, color='C0', width=width)
    plt.grid()
    plt.show()

In [ ]:
result = superposition()
plot_amplitudes(result)

### Coding the oracle
The oralcle is the operator that changes the sign of the amplitudes of the quantum states that encode solutions of the problem.


In [ ]:
def create_oracle(state='111'):
    """Oracle"""
    nqubits = len(state)
    index = [i for i,value in enumerate(list(state)) if value == '0']
    oracle = models.Circuit(nqubits+1)
    oracle.add([gates.X(i) for i in index])
    oracle.add(gates.X(nqubits).controlled_by(*range(nqubits)))
    oracle.add([gates.X(i) for i in index])
    return oracle

Note the use of `gate.controlled_by` method which allows to control any gate to an arbitrary number of qubits

In [ ]:
oracle = create_oracle('101')
print(oracle.draw())

Lets check again the final amplitudes after creating the superposition and applying the oracle. 

First we create the total circuit

In [ ]:
circuit = superposition + oracle
print(circuit.draw())

and then we plot the final amplitudes using the functon `plot_amplitude`

In [ ]:
result = circuit()
plot_amplitudes(result)

Notice how the oracle inverted the sign of the amplitude of the target state.

### Coding the diffuser
To perform the diffusion operator we need to invert anything perpendicular to |s⟩. This can be done using a method similar to the Oracle.


In [ ]:
def create_diffuser(nqubits):
    diffuser = models.Circuit(nqubits)
    for i in range(nqubits):
        diffuser.add(gates.H(i))
    for i in range(nqubits):
        diffuser.add(gates.X(i))
    diffuser.add(gates.Z(0).controlled_by(*range(1,nqubits)))
    for i in range(nqubits):
        diffuser.add(gates.X(i))
    for i in range(nqubits):
        diffuser.add(gates.H(i))
    return diffuser

Let's check the amplitudes after the diffusion using the same methods as above

In [ ]:
diffuser = create_diffuser(3)
circuit = superposition + oracle
circuit.add(diffuser.on_qubits(*range(3)))
print(circuit.draw())

In [ ]:
result = circuit()
plot_amplitudes(result)

Notice how the diffuser increased the amplitude of the target state.

### Grover's algorithm

Repeating the oracle + diffuser operation for many iterations, further amplifies the probability to measure the target state.

In [ ]:
def create_grover(state, iterations):
    """Complete circuit that implements Grover's algorithm.
    
    Args:
        state (str): Target state.
        iterations (int): Number of times the oracle + diffuser operation is repeated.
    """
    nqubits = len(state)
    superposition = create_superposition(nqubits)
    oracle = create_oracle(state)
    diffuser = create_diffuser(nqubits)
    
    grover = models.Circuit(nqubits+1)
    grover += superposition
    for _ in range(iterations):
        grover += oracle
        grover.add(diffuser.on_qubits(*range(nqubits)))
    # measure all qubits
    grover.add([ gates.M(i) for i in range(nqubits)])
    return grover

You can check how the amplitude is affected by changing the number of iterations below

In [ ]:
grover = create_grover('101', 2)
plot_amplitudes(grover())

The number of iterations that gives the highest probability is given by

$$ \frac{\pi }{4}\sqrt{\frac{2^{n_{qubits}}}{n_{solutions}}} $$

where $n_{solutions}$ is the number of target solutions to the search. In our case $n_{solutions} = 1$.

In [ ]:
def grover_iterations(nqubits, nsol=1):
    return int((np.pi/4)*np.sqrt((2**nqubits)/nsol))

In [ ]:
iterations = grover_iterations(3)
iterations

In [ ]:
grover = create_grover('101', iterations)
plot_amplitudes(grover())

We can also have a look at the frequencies of measuring each bitstring

In [ ]:
result = grover(nshots=1000)
result.frequencies()

## Let's try to run the Grover's algorithm with an increasing number of qubits?

Lets run the following benchmark to understand why hardware acceleration is important.

In [ ]:
import time

def performance(backend, qubit_range):
    set_backend(backend)
    
    for i in qubit_range:
        iterations = grover_iterations(i,1)
        print("nqubits", i, end="")
        state = "1" * i
        circuit = create_grover(state, iterations)
        start = time.time()
        result = circuit(nshots=1000)
        end = time.time()
        print(f"\tTime = {end-start}")
        # Frequency of the target bitstring
        freq = result.frequencies().get(i * '1')
        print("Frequency =", freq)
        print()

In [ ]:
performance("numpy", range(4,16))

In [ ]:
performance("qibojit", range(4,16))